**Commentaire:** _Nous avons choisi ici d'entraîner le modèle sur les données brutes sans traitement préalable des textes des tweets._

I. Importation des packages nécessaires

II. Importation du jeu de données

III. Séparation des données

VI. Entraînement des modèles

> 1. Régression logistique

> 2. Naif bayésien BernoulliNB

> 3. Naif bayésien MultinomialNB

> 4. K plus proches voisins

> 5. Arbre de décision

> 6. Forêt aléatoire

> 7. XGBOOST

> 8. Gradient Boosting

> 9. Adaboost

# PATH

In [4]:
#For Colab
# from google.colab import drive
# drive.mount('/content/gdrive')
# file_path = 'gdrive/My Drive/Colab Notebooks/SentimentAnalysisTwitter/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
#For local
file_path = ''

On enregistre les modèles entraînés

In [3]:
trained_models_path = file_path + 'Best_models_A/'

# I. Importation des packages nécessaires

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize, pos_tag, sent_tokenize
from sklearn.metrics import classification_report
import pickle
import os

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/PhuocNhatDANG/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/PhuocNhatDANG/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/PhuocNhatDANG/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# II. Importation du jeu de données 

In [5]:
data_path = os.path.split(os.getcwd())[0]+'/data/data_clean'

In [6]:
df = pd.read_csv(data_path +'/data_clean.csv')

In [7]:
df.head()

,index,target,ids,date,flag,user,text,clean_text,clean_text_tagged,clean_text_tagged_bis
0,0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot awww bummer shoulda got david carr ...,"[('switchfoot', 'NN'), ('awww', 'NN'), ('bumme...","('switchfoot', 'NN'), ('awww', 'NN'), ('bummer..."
1,1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...,"[('upset', 'JJ'), ('updat', 'JJ'), ('facebook'...","('upset', 'JJ'), ('updat', 'JJ'), ('facebook',..."
2,2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save 50 res...,"[('kenichan', 'NNS'), ('dive', 'VBP'), ('mani'...","('kenichan', 'NNS'), ('dive', 'VBP'), ('mani',..."
3,3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire,"[('whole', 'JJ'), ('bodi', 'NN'), ('feel', 'VB...","('whole', 'JJ'), ('bodi', 'NN'), ('feel', 'VB'..."
4,4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see,"[('nationwideclass', 'NN'), ('behav', 'NN'), (...","('nationwideclass', 'NN'), ('behav', 'NN'), ('..."


# III. Séparation des données

In [8]:
X = df['text']
y = df.target

In [9]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.9, random_state = 0)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [9]:
# Enregistrement
pickle.dump(vectorizer, open(trained_models_path+'vectorizer.sav', 'wb'))

In [11]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(100), max_iter=83, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=21,tol=0.000000001)

In [12]:
from time import time
t0 = time() 
clf.fit(X_train, y_train)
t1 = time() - t0
print("Réalisé en {} secondes".format(round(t1,3)))
# pickle.dump(clf, open(file_path+'mlp_cvec_400.sav', 'wb'))
# y_pred = clf.predict(X_test)

Iteration 1, loss = 0.69374965
Réalisé en 16.616 secondes


/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [13]:
y_pred = clf.predict(X_test)

In [14]:
clf.best_loss_

0.6937496499326332

# VI. Entraînement des modèles

## 1. Régression logistique 

In [1]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=0.1, penalty ='l2')
logreg.fit(X_train, y_train)

In [0]:
# Enregistrement
pickle.dump(logreg, open(trained_models_path+'logreg.sav', 'wb'))

### Avec une TF-IDF

In [0]:
X_train_tf,X_test_tf,y_train_tf,y_test_tf = train_test_split(X,y,test_size=0.2,random_state = 0)

In [0]:
tvec = TfidfVectorizer()
X_train_tvec = tvec.fit_transform(X_train_tf)
X_test_tvec = tvec.transform(X_test_tf)

In [0]:
# Enregistrement
pickle.dump(tvec, open(trained_models_path+'tvec.sav', 'wb'))

In [0]:
logreg_tfid = linear_model.LogisticRegression(C=0.1, penalty ='l2').fit(X_train_tvec, y_train_tf)

In [0]:
# Enregistrement
pickle.dump(logreg_tfid, open(trained_models_path+'logreg_tfid.sav', 'wb'))

## 2. Naif bayésien BernoulliNB

In [2]:
from sklearn.naive_bayes import BernoulliNB
ber = BernoulliNB().fit(X_train, y_train)

# Enregistrement
pickle.dump(ber, open(trained_models_path+'ber.sav', 'wb'))

##  3. Naif bayésien MultinomialNB


In [0]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train, y_train)

# Enregistrement
pickle.dump(nb, open(trained_models_path+'nb.sav', 'wb'))

## 4. K plus proches voisins

In [4]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors=13, metric= "minkowski", weights = 'distance').fit(X_train,y_train)

# Enregistrement
pickle.dump(knn, open(trained_models_path+'knn.sav', 'wb'))

## 5. Arbre de décision 

In [7]:
from sklearn.tree import DecisionTreeClassifier
dc = DecisionTreeClassifier().fit(X_train,y_train)

# Enregistrement
pickle.dump(dc, open(trained_models_path+'dc.sav', 'wb'))

## 6. Forêt aléatoire


In [6]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier(n_jobs = -1).fit(X_train,y_train)

# Enregistrement
pickle.dump(rf, open(trained_models_path+'rf.sav', 'wb'))

## 7. XGBOOST

In [8]:
from xgboost import XGBClassifier
xgb = XGBClassifier().fit(X_train, y_train)

# Enregistrement
pickle.dump(xgb, open(trained_models_path+'xgb.sav', 'wb'))

## 8. Gradient Boosting

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
gradbt = GradientBoostingClassifier().fit(X_train, y_train)

# Enregistrement
pickle.dump(gradbt, open(trained_models_path+'gradbt.sav', 'wb'))

## 9. Adaboost

In [10]:
#Adaboost
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier 
dtc_u = DecisionTreeClassifier(max_depth = 3)
ac = AdaBoostClassifier(base_estimator = dtc_u,learning_rate = 0.1, n_estimators = 400)
ac.fit(X_train,y_train)

# Enregistrement
pickle.dump(ac, open(trained_models_path+'ac.sav', 'wb'))

## 10. MLP

In [12]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=50, alpha=0.0001, solver='sgd', verbose=10,  random_state=21,tol=0.000000001).fit(X_train, y_train)

# Enregistrement
pickle.dump(mlp, open(trained_models_path+'mlp.sav', 'wb'))

Iteration 1, loss = 0.68024771
Iteration 2, loss = 0.65015452


/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
